In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


In [4]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop('EIN', axis=1)
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
application_df = application_df.drop('NAME', axis=1)
application_df.head(10)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
6,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,31452,1
7,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,7508025,1
8,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,94389,1
9,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [7]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Determine which values to replace if counts are less than ...?
replace_application = app_counts[app_counts < 66].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# Determine which values to replace if counts are less than ..?
replace_class = classification_counts[classification_counts < 1000].index.tolist()

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Merge one-hot encoded features and drop the originals
application_df_encoded = application_df.merge(encode_df,left_index=True, right_index=True).drop(columns=application_cat, axis=1)
application_df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
# Split our preprocessed data into our features and target arrays
y = application_df_encoded["IS_SUCCESSFUL"].values
X = application_df_encoded.drop(["IS_SUCCESSFUL"], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78) 

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second third layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 80)                3680      
                                                                 
 dense_11 (Dense)            (None, 30)                2430      
                                                                 
 dense_12 (Dense)            (None, 10)                310       
                                                                 
 dense_13 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6,431
Trainable params: 6,431
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5696 - accuracy: 0.7222
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5551 - accuracy: 0.7290
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5518 - accuracy: 0.7310
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5503 - accuracy: 0.7331
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5498 - accuracy: 0.7334
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5483 - accuracy: 0.7334
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5478 - accuracy: 0.7345
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5476 - accuracy: 0.7338
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5462 - accuracy: 0.7332
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5470 - accura

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5537 - accuracy: 0.7247 - 867ms/epoch - 3ms/step
Loss: 0.5537338256835938, Accuracy: 0.7246647477149963


In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 40
units=hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second third layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 100)               4600      
                                                                 
 dense_15 (Dense)            (None, 40)                4040      
                                                                 
 dense_16 (Dense)            (None, 10)                410       
                                                                 
 dense_17 (Dense)            (None, 1)                 11        
                                                                 
Total params: 9,061
Trainable params: 9,061
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5713 - accuracy: 0.7185
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5537 - accuracy: 0.7304
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5522 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5494 - accuracy: 0.7331
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5494 - accuracy: 0.7343
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5485 - accuracy: 0.7323
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5470 - accuracy: 0.7348
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5471 - accuracy: 0.7340
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5468 - accuracy: 0.7355
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5465 - accura

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5612 - accuracy: 0.7261 - 851ms/epoch - 3ms/step
Loss: 0.561190128326416, Accuracy: 0.726064145565033


In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 40
units=hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Second third layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 100)               4600      
                                                                 
 dense_20 (Dense)            (None, 40)                4040      
                                                                 
 dense_21 (Dense)            (None, 10)                410       
                                                                 
 dense_22 (Dense)            (None, 1)                 11        
                                                                 
Total params: 9,061
Trainable params: 9,061
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5655 - accuracy: 0.7240
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5548 - accuracy: 0.7299
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5514 - accuracy: 0.7324
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5493 - accuracy: 0.7321
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5485 - accuracy: 0.7327
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5469 - accuracy: 0.7341
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5467 - accuracy: 0.7336
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5459 - accuracy: 0.7350
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5453 - accuracy: 0.7353
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5440 - accura

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5562 - accuracy: 0.7265 - 875ms/epoch - 3ms/step
Loss: 0.5562031865119934, Accuracy: 0.7265306115150452


In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 60
hidden_nodes_layer3 = 30
hidden_nodes_layer4 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 100)               4600      
                                                                 
 dense_24 (Dense)            (None, 60)                6060      
                                                                 
 dense_25 (Dense)            (None, 30)                1830      
                                                                 
 dense_26 (Dense)            (None, 10)                310       
                                                                 
 dense_27 (Dense)            (None, 1)                 11        
                                                                 
Total params: 12,811
Trainable params: 12,811
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 5s 3ms/step - loss: 0.5313 - accuracy: 0.7421
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5309 - accuracy: 0.7421
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5310 - accuracy: 0.7423
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5309 - accuracy: 0.7427
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5313 - accuracy: 0.7426
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5309 - accuracy: 0.7418
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5305 - accuracy: 0.7423
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5311 - accuracy: 0.7432
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5312 - accuracy: 0.7423
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5313 - accura

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5605 - accuracy: 0.7266 - 1s/epoch - 4ms/step
Loss: 0.5604853630065918, Accuracy: 0.7266472578048706


In [38]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  200
hidden_nodes_layer2 = 120
hidden_nodes_layer3 = 60
hidden_nodes_layer4 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 200)               9200      
                                                                 
 dense_29 (Dense)            (None, 120)               24120     
                                                                 
 dense_30 (Dense)            (None, 60)                7260      
                                                                 
 dense_31 (Dense)            (None, 30)                1830      
                                                                 
 dense_32 (Dense)            (None, 1)                 31        
                                                                 
Total params: 42,441
Trainable params: 42,441
Non-trainable params: 0
_________________________________________________________________


In [39]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 6s 5ms/step - loss: 0.5677 - accuracy: 0.7217
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5569 - accuracy: 0.7287
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5537 - accuracy: 0.7295
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5515 - accuracy: 0.7312
Epoch 5/100
804/804 [==============================] - 4s 6ms/step - loss: 0.5502 - accuracy: 0.7312
Epoch 6/100
804/804 [==============================] - 4s 6ms/step - loss: 0.5493 - accuracy: 0.7331
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5479 - accuracy: 0.7336
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5474 - accuracy: 0.7348
Epoch 9/100
804/804 [==============================] - 4s 6ms/step - loss: 0.5458 - accuracy: 0.7342
Epoch 10/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5452 - accura

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5561 - accuracy: 0.7255 - 1s/epoch - 4ms/step
Loss: 0.5560784935951233, Accuracy: 0.7254810333251953


In [41]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("AlphabetSoupCharity_Optimization.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=5)
checkpoint_model = nn.fit(X_train, y_train,
          epochs=100,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=[checkpoint])

Epoch 1/100
804/804 [==============================] - 5s 6ms/step - loss: 0.6980 - accuracy: 0.5275 - val_loss: 0.6927 - val_accuracy: 0.5332
Epoch 2/100
804/804 [==============================] - 5s 6ms/step - loss: 0.6918 - accuracy: 0.5272 - val_loss: 0.6910 - val_accuracy: 0.5332
Epoch 3/100
804/804 [==============================] - 5s 6ms/step - loss: 0.6918 - accuracy: 0.5308 - val_loss: 0.6911 - val_accuracy: 0.5332
Epoch 4/100
804/804 [==============================] - 5s 6ms/step - loss: 0.6917 - accuracy: 0.5304 - val_loss: 0.6913 - val_accuracy: 0.5332
Epoch 5/100
801/804 [============================>.] - ETA: 0s - loss: 0.6916 - accuracy: 0.5302
Epoch 5: loss improved from inf to 0.69160, saving model to AlphabetSoupCharity_Optimization.h5
804/804 [==============================] - 5s 7ms/step - loss: 0.6916 - accuracy: 0.5305 - val_loss: 0.6910 - val_accuracy: 0.5332
Epoch 6/100
804/804 [==============================] - 5s 6ms/step - loss: 0.6918 - accuracy: 0.5279 - v

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5589 - accuracy: 0.7247 - 764ms/epoch - 3ms/step
Loss: 0.558869481086731, Accuracy: 0.7246647477149963
